In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Sentiment Analysis

In [3]:
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
session_token = os.getenv("AWS_SESSION_KEY")
region_name = os.getenv("AWS_DEFAULT_REGION")


In [4]:
region_name = 'us-west-2'
bedrock_client = boto3.client('bedrock-runtime', region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  )


In [5]:
# Fonction pour l'analyse de sentiment en utilisant l'API Messages de Claude 3
def analyze_sentiment(text):
    try:
        # Préparation de la requête pour l'API Messages
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 50,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Analyse le sentiment de ce texte : \"{text}\""}
                ]
            })
        )

        # Affichage de la réponse brute pour diagnostic
        response_body = json.loads(response['body'].read().decode('utf-8'))
        
        # Vérification de la structure de la réponse et extraction du texte
        content = response_body.get("content")
        if isinstance(content, list):
            sentiment_analysis = " ".join([part["text"] for part in content if "text" in part])
        else:
            sentiment_analysis = content  # Directement utiliser le contenu si c'est une chaîne
        
        return sentiment_analysis or "Pas de résultat détecté"
    except Exception as e:
        print("Erreur :", e)
        return None

In [6]:
# # Exemple d'utilisation
# text_to_analyze = "J'aime pas le fromage."
# result = analyze_sentiment(text_to_analyze)
# print("Analyse de sentiment :", result)

## RAG Agent

In [7]:
agent_id = os.getenv("AGENT_ID")
agent_alias = os.getenv("AGENT_ALIAS")

In [8]:
runtime_client=boto3.client(
    service_name="bedrock-agent-runtime",
    region_name=region_name,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    aws_session_token=session_token  # Facultatif
),


In [9]:
runtime_client[0]

In [10]:
import uuid
session_id = str(uuid.uuid4())

# Function to call the agent with a user prompt
def call_agent_with_prompt(user_prompt):
    try:
        # Invoke the agent without invalid parameters
        response = runtime_client[0].invoke_agent(
            agentId=agent_id,
            agentAliasId=agent_alias,  # Include this if you have an agent alias
            sessionId=session_id,
            inputText=user_prompt,
            #enableTrace=True
        )
        event_stream = response.get('completion')
        response_text = ''

        # Parcourir l'EventStream pour obtenir la réponse
        for event in event_stream:
            # Chaque événement est un dictionnaire contenant les données de l'événement
            # Pour la génération de texte, les données sont sous la clé 'chunk'
            if 'chunk' in event:
                chunk = event['chunk']
                # La clé 'bytes' contient le texte généré
                content = chunk.get('bytes', b'').decode('utf-8')
                response_text += content
            else:
                print("Événement non traité :", event)

        return response_text

    except Exception as e:
        print("Erreur lors de l'appel à l'agent :", e)
        return None

In [11]:
# # Exemple d'appel avec un *prompt* utilisateur
# user_prompt = "Peux tu analyser une analyse fondamentale de Accenture ?"
# response = call_agent_with_prompt(user_prompt)
# print("Réponse de l'agent :", response)


## Agent + macro + sentiment

In [12]:
import yfinance as yf
import pandas as pd

In [13]:
ticker = yf.Ticker("AAPL")

Sentiment

In [14]:
import requests
from bs4 import BeautifulSoup
import json

# Fonction pour extraire le texte principal de l'article
def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraction des paragraphes de l'article
    paragraphs = soup.find_all('p')
    article_text = ' '.join([para.get_text() for para in paragraphs])
    return article_text

In [15]:

def analyze_subject_sentiment(text):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 100,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Donne uniquement sujet principal de cet article ainsi que le sentiment du text associé sans explicatioN: \"{text}\""}
                ]
            })
        )
        
        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        subject_analysis = content[0]["text"] if content else "Pas de sujet détecté"
        return subject_analysis
    
    except Exception as e:
        print("Erreur d'analyse du sujet :", e)
        return None



In [16]:

# Fonction principale pour obtenir les informations d'actualité avec sujet et sentiment
def get_news_with_sentiment(ticker):
    

    news = ticker.news
    sentiment_output = ''
    
    for item in news:
        title = item.get('title')
        link = item.get('link')
        source = item.get('publisher')
        
        # Extraire le texte de l'article
        article_text = extract_article_text(link)
        
        # Analyser le sujet et le sentiment
        subject_sentiment = analyze_subject_sentiment(article_text)
        
        sentiment_output += title + subject_sentiment
        

    return sentiment_output

In [17]:
sentiment_output = get_news_with_sentiment(ticker)

In [18]:
sentiment_output

'Buffett Keeps Cutting Apple Stake, With Value Down 60% This YearSujet principal: Ventes d\'actions Apple par Berkshire Hathaway\n\nSentiment: Neutre3 Things Apple Doesn\'t Want to Talk AboutSujet principal: Problème technique en cours de résolution\n\nSentiment: RassurantBig Pension Sold Palantir, Apple, and Nvidia Stock. It Bought AMD.Sujet principal: Problème technique en cours de résolution\n\nSentiment: Rassurant, positifApple Intelligence Products Could Make Up For Apple’s (AAPL) iPhone WeaknessLe sujet principal de l\'article semble être les perspectives d\'Apple dans la course à l\'intelligence artificielle, en particulier avec le lancement imminent de son premier appareil "Apple Intelligence".\n\nLe sentiment général du texte est mitigé, reconnaissant les défis actuels d\'Apple comme les ventes d\'iPhone décevantes et son retard dans la course à l\'IA, mais restant optimiste quant au potPixelmator announces agreement to be acquired by AppleSujet principal: Acquisition de Pixel

Macro

In [19]:
def caption_summary(text,max_token):
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            contentType="application/json",
            accept="application/json",
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_token,
                "temperature": 0.5,
                "messages": [
                    {"role": "user", "content": f"Tu es un macro économiste, rédige un résumé des points clés  macro économique des différents sujets, avec détails (ex chiffres) : \"{text}\""}
                ]
            })
        )

        response_body = json.loads(response['body'].read().decode('utf-8'))
        content = response_body.get("content")
        
        sentiment_analysis = content[0]["text"] if content else "Pas de sentiment détecté"
        return sentiment_analysis
    
    except Exception as e:
        print("Erreur d'analyse de sentiment :", e)

In [20]:

def get_macro_news() :
    tickers = {
        "^GSPC": "S&P 500 - Indice Boursier",
        "DX-Y.NYB": "Indice du Dollar Américain",
        "^TNX": "Taux des Obligations à 10 ans",
        "GC=F": "Prix de l'Or",
        "CL=F": "Prix du Pétrole (WTI)",
        "^VIX": "VIX - Volatilité des Marchés"
    }
    
    output_text = ""
    for ticker_symbol, description in tickers.items():
        ticker = yf.Ticker(ticker_symbol)
        news_items = ticker.news
        
        for item in news_items[:2]:
            
            text_article = extract_article_text(item["link"])
            if text_article != "Thank you for your patience. Our engineers are working quickly to resolve the issue." :
                text = caption_summary(text_article,150)
                
                output_text += text
    caption_macro = caption_summary(output_text,max_token=300)
    return caption_macro

In [21]:
output_macro = get_macro_news()

In [22]:
output_macro

"Voici un résumé des principaux points macroéconomiques avec des détails chiffrés :\n\nÉlection présidentielle américaine 2024 :\n- Le S&P 500 a progressé de 20% jusqu'à présent cette année, faisant de 2024 la meilleure année électorale jusqu'en octobre depuis 1936. \n- Les marchés anticipent une forte volatilité en raison de la course serrée entre les candidats.\n- Le marché des prédictions Polymarket montre une chance de 59,5% pour que Donald Trump remporte l'élection.\n\nImpacts économiques du conflit en Ukraine :\n- Le prix du baril de pétrole Brent a dépassé les 100$ en raison des perturbations d'approvisionnement liées au conflit.\n- L'inflation aux États-Unis a atteint 8,5% sur un an en mars 2022, son plus haut niveau depuis 40 ans.\n- La Fed a relevé ses taux directeurs de 0,75 point de pourcentage en juin, sa plus forte hausse depuis 1994.\n\nMarchés des métaux précieux :  \n- L'or évoluait autour d'un niveau record de"

In [23]:
# Exemple d'appel avec un *prompt* utilisateur
user_prompt = "Réalise le rapport sur AAPL en détaillant les 5 parties ?"
promt = user_prompt + "Context macro suppélementaire :" + output_macro + "Sujets et sentiments actuelles autour de l'entreprise :" + sentiment_output
response = call_agent_with_prompt(promt)
print("Réponse de l'agent :", response)

Réponse de l'agent : Voici un rapport détaillé en 5 parties sur l'entreprise Apple Inc. (AAPL) :

1. Présentation de l'entreprise :
Apple Inc. est une entreprise américaine spécialisée dans la conception, la fabrication et la commercialisation de smartphones, d'ordinateurs personnels, de tablettes, de produits connectés et d'accessoires. Basée à Cupertino en Californie, Apple emploie environ 150 000 personnes dans le monde. L'entreprise opère dans le secteur de la technologie et fait partie de l'indice S&P 500.

2. Indicateurs financiers clés :
- Chiffre d'affaires : Apple a généré un chiffre d'affaires de 7,4 à 8,1 milliards de dollars par trimestre en 2023-2024.
- EBITDA ajusté : L'EBITDA ajusté d'Apple se situe entre 492,5 et 723 millions de dollars par trimestre.
- Bénéfice net ajusté : Le bénéfice net ajusté d'Apple varie entre 125,7 et 261 millions de dollars par trimestre.
- Dividende : Apple verse un dividende trimestriel d'environ 0,63 dollar par action, avec un ratio de distr